In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-13 02:21:58.421 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

data_sets = data_sets_factory(**config["data_set"])
logger.info("Loaded {} data sets.", len(data_sets))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = [transform(data_set) for data_set in data_sets]

dataset = data_sets[0]

2024-12-13 02:21:58.427 | INFO     | __main__:<module>:3 - Loading data.
2024-12-13 02:21:58.427 | INFO     | __main__:<module>:6 - Loaded 1 data sets.
2024-12-13 02:21:58.427 | INFO     | __main__:<module>:10 - Start preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [7]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset,batch_size=1)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    dim=len(train_dataset[0]["branch_lengths"]),
    **config["training"]["model_config"],
)

In [8]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_simple_conditional_flow/epoch=46-val_loss=-3978.27-v1.ckpt")

In [9]:
sample = next(iter(test_loader))
sample

{'branch_lengths': tensor([[-8.5946e-05, -1.0166e-04, -2.9394e-04,  1.0320e-03, -1.3393e-03,
          -9.7662e-04,  9.8861e-04,  5.5989e-04, -1.6961e-03, -3.0541e-04]]),
 'clades_one_hot': tensor([[1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
          0., 1.]])}

In [10]:
latent = model.forward(sample)
latent

{'z': tensor([[-0.6564, -1.1955, -1.4612,  0.0499,  1.4594, -0.3294,  1.3576, -1.6930,
          -0.3948,  1.4051]], grad_fn=<MulBackward0>),
 'context': tensor([[1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
          0., 1.]]),
 'log_dj': tensor([31.9492], grad_fn=<AddBackward0>)}

In [21]:
latent["z"] = torch.distributions.normal.Normal(loc=0.0, scale=1.0).sample((10,))
output = model.inverse(latent)
output

{'branch_lengths': tensor([-7.3658,  0.8086,  2.1286,  2.2383, -1.8624, -0.7282, -3.7896, -0.3223,
         -0.7825, -1.1896], grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
          0., 1.]]),
 'log_dj': tensor([31.9492], grad_fn=<AddBackward0>)}